### Industrial Production

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Retrieve latest data if update available
url_base = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17'
dates = 'from=01/01/1988&to=12/31/2023'
series = 'series=f97ad3652d87a6c1720943c31423103a'
settings = 'filetype=csv&label=include&layout=seriescolumn'
url = f'{url_base}&{series}&lastobs=&{dates}&{settings}'

d, data = clean_fed_data(url)

pd.to_pickle(d,data_dir / 'indpro.pickle')
data = data.rename(d, axis=1)
data.to_csv(data_dir / 'indpro_raw.csv', index_label='date', float_format='%g')
(data.loc['1989':, ['Manufacturing', 'Total index']]
 .to_csv(data_dir / 'indpro.csv', index_label='date', float_format='%g'))

print('Latest: ', dtxt(data.index[-1])['mon1'])

Latest:  October 2023


In [3]:
# Retrieve raw data from csv
data = pd.read_csv(data_dir / 'indpro_raw.csv', 
                   index_col='date', parse_dates=True)
d = pd.read_pickle(data_dir / 'indpro.pickle')
# Retrieve weights (relative importance)
adj_series_dict = {k[3:-2]: v for k, v in d.items()}
series = adj_series_dict.keys()
url = 'https://www.federalreserve.gov/releases/g17/ipdisk/ipweights_sa.txt'
columns = ['Series', 'Year', 'January', 'February', 'March', 
          'April', 'May', 'June', 'July', 'August', 
          'September', 'October', 'November', 'December']
r = requests.get(url)
raw_weights = pd.read_csv(io.StringIO(r.content.decode('utf-8')), 
                          sep='\s+', skiprows=1)
raw_weights.columns = columns
weights = (raw_weights[raw_weights['Series'].isin(series)]
           .set_index(['Series', 'Year']).stack().reset_index())
weights['Date'] = (pd.to_datetime(weights['level_2'] + ' 01, ' 
                + weights['Year'].astype('int').astype('str')))
weights = (weights.set_index(['Series', 'Date'])[0]
           .unstack().T.rename(adj_series_dict, axis=1)
           .loc['1988':])
weights['ENS'] = (weights['Equipment, total'] + 
                  weights['Nonindustrial supplies'])

# Apply calculations
growth = data.pct_change(12).dropna() * 100
growth['ENS'] = ((growth['Equipment, total'] * 
                  (weights['Equipment, total'] / weights['ENS']))
                 + (growth['Nonindustrial supplies'] * 
                    (weights['Nonindustrial supplies'] / weights['ENS'])))
contrib = growth / 100 * weights.iloc[12:]
contrib['Non-energy materials'] = contrib['Materials'] - contrib['Energy materials']
contrib['ENS'] = contrib['Equipment, total'] + contrib['Nonindustrial supplies']

srs = ['Consumer goods', 'ENS', 'Materials', 'Durable manufacturing', 'Energy materials',
       'Mining', 'Nondurable manufacturing', 'Electric and gas utilities', 
       'Non-energy materials']

(contrib[srs].resample('QS').mean()
     .to_csv(data_dir / 'indprogr.csv', index_label='date', float_format='%g'))

ltd = contrib.loc[:, srs].iloc[-24:]  # subset of most recent monthly data
ltd['label'] = [dt.strftime('%b\\\%Y') if dt.month == 1 
                  else dt.strftime('%b') if dt.month in [7]
                  else '' for dt in ltd.index]

(ltd.to_csv(data_dir / 'indprogr_rec.csv', 
            index_label='date', float_format='%g'))

In [4]:
# Text 1 
ltmo = dtxt(data.index[-1])['mon1']
ltmosh = ltmo[:-5]
prmo = dtxt(data.index[-2])['mon1']
prmo = prmo[:-5] if ltmo[-5:] == prmo[-5:] else prmo
ltgr, prgr = growth.iloc[-1], growth.iloc[-2]
ltco, prco = contrib.iloc[-1], contrib.iloc[-2]

tot = value_text(ltgr['Total index'])
prtot = value_text(prgr['Total index'], 'increase_of')
man = value_text(ltgr['Manufacturing'])
manco = value_text(ltco['Manufacturing'], 'contribution_to', 'pp', threshold=0.1)
minco = value_text(ltco['Mining'], 'contribution', 'pp', threshold=0.1)
util_t = 'Electric and gas utilities'
utilco = value_text(ltco[util_t], 'contribution', 'pp', threshold=0.1)
cg = value_text(ltco['Consumer goods'], 'contribution_to', 'pp', threshold=0.1)
eqp = value_text(ltco['Equipment, total'], 'contribution', 'pp', threshold=0.1)
sup = value_text(ltco['Nonindustrial supplies'], 'contribution', 'pp', threshold=0.1)
mat = value_text(ltco['Materials'], 'contribution', 'pp', threshold=0.1)

# Text 1 (overview)
ip_url = 'https://www.federalreserve.gov/releases/g17/Current/default.htm'
txt1 = ('The Federal Reserve industrial production index '+
        f'\href{{{ip_url}}}{{measures}} the real output of '+
        'the industrial sector, which includes manufacturing, '+
        'mining, and electric and gas utilities.\n\n'+
        f'\\textbf{{Industrial production}} {tot} over the year '+
        f'ending {ltmo}, following {prtot} in {prmo}. The manufacturing-'+
        f'only index {man} over the year ending {ltmosh}, and {manco} '+
        f'the growth of the total index. Mining {minco}, and utilities '+
        f'{utilco}.\n\nBy market group, finished consumer goods {cg} '+
        f'one-year industrial production growth in {ltmosh}. Business '+
        f'equipment {eqp}, nonindustrial supplies {sup}, '+
        f'and materials {mat}.')
write_txt(text_dir / 'indpro.txt', txt1)
print(txt1)

The Federal Reserve industrial production index \href{https://www.federalreserve.gov/releases/g17/Current/default.htm}{measures} the real output of the industrial sector, which includes manufacturing, mining, and electric and gas utilities.

\textbf{Industrial production} decreased 0.7 percent over the year ending October 2023, following a decrease of 0.2 percent in September. The manufacturing-only index decreased 1.7 percent over the year ending October, and subtracted 1.3 percentage points from the growth of the total index. Mining contributed 0.3 percentage point, and utilities contributed 0.3 percentage point.

By market group, finished consumer goods subtracted 0.5 percentage point from one-year industrial production growth in October. Business equipment subtracted 0.1 percentage point, nonindustrial supplies subtracted 0.3 percentage point, and materials contributed 0.3 percentage point.


In [9]:
# Text 2 (Recent in detail)
ltd = ltd.drop('label', axis=1)
lt = ltd.iloc[-1].sum() / 2
pr = ltd.iloc[-2].sum() / 2
avg = ltd.mean().sum() / 2
rec1_txt = compare_text(lt, avg, [0.5, 2.0, 5.0])
rec2_txt = compare_text(lt, pr, [0.5, 2.0, 5.0])

text0 = ('Looking more closely at recent industrial production growth, '+
        f'the latest one-year growth rate, covering {ltmo}, is {rec1_txt} '+
        f'the five-year average, and {rec2_txt} the {prmo} growth rate. ')

df = pd.read_csv(data_dir / 'indprogr_rec.csv', index_col='date', parse_dates=True)
# Market group
srs1 = ['Consumer goods', 'ENS', 'Energy materials', 'Non-energy materials']
data1 = df[srs1]
# Adjust columns for use in text
data1 = data1.rename({'ENS': 'Business equipment and non-industrial supplies'}, axis=1)
data1.columns = data1.columns.str.lower()
mu, sigma = 0.5, 2
text1, bbdb1 = gc_desc(data1.iloc[-1], mu, sigma)

# Industry group
srs2 = ['Durable manufacturing', 'Nondurable manufacturing', 
        'Mining', 'Electric and gas utilities']
data2 = df[srs2]
data2.columns = data2.columns.str.lower() # Adjust columns for use in text
text2, bbdb2 = gc_desc(data2.iloc[-1], mu, sigma)
if text1 == text2:
    text = (f'By both market group and industry group, the latest {text1}')
elif (bbdb1 == bbdb2) & (text1 != text2):
    text2, bbdb2 = gc_desc(data2.iloc[-1], mu, sigma, also=True)
    text = (f'By market group, the latest {text1} By industry group, the latest {text2}')
else:
    text = (f'By market group, the latest {text1} By industry group, the latest {text2}')
para = '\n\n' if len(text) > 160 else ''
final = f'{text0}{para}{text}'
print(final)
write_txt(text_dir / 'indpro_rec.txt', final)

Looking more closely at recent industrial production growth, the latest one-year growth rate, covering October 2023, is substantially below the five-year average, and slightly below the September growth rate. 

By market group, the latest decrease is driven by a decrease in consumer goods and business equipment and non-industrial supplies, and partially offset by an increase in energy materials. By industry group, the latest decrease is also driven by a decrease in nondurable manufacturing and durable manufacturing.


### IP Table

In [10]:
n = {'Total index': 'Total Index',
     'Manufacturing': '\hspace{2mm}Manufacturing',
     'Durable manufacturing': '\hspace{-3mm} \cbox{blue!62!black} Durable Manufacturing',
     'Motor vehicles and parts': '\hspace{5mm}Motor Vehicles \& Parts',
     'Nondurable manufacturing': '\hspace{-3mm} \cbox{blue!20!cyan!80!white} Nondurable Manufacturing',
     'Mining': '\hspace{-3mm} \cbox{orange!30!yellow} Mining',
     'Electric and gas utilities': '\hspace{-3mm} \cbox{green!70!blue} Utilities',
     'Consumer goods': '\hspace{-3mm} \cbox{violet!70!purple!90!black} Consumer Goods',
     'Durable consumer goods': ' \hspace{4mm}Consumer Durables',
     'Automotive products': ' \hspace{6mm}Automotive Products',
     'Nondurable consumer goods': ' \hspace{4mm}Consumer Nondurables',
     'Foods and tobacco': ' \hspace{6mm}Foods \& Tobacco',
     'Chemical products': ' \hspace{6mm}Chemical Products',
     'Consumer energy products': ' \hspace{6mm}Consumer Energy Products',
     'ENS': '\hspace{-3mm} \cbox{magenta} Business Equipment \& Supplies',
     'Equipment, total': '\hspace{4mm}Equipment',
     'Industrial equipment': ' \hspace{6mm}Industrial Equipment',
     'Nonindustrial supplies': ' \hspace{4mm}Nonindustrial Supplies',
     'Construction supplies': ' \hspace{6mm}Construction Supplies',
     'Business supplies': ' \hspace{6mm}Business Supplies',
     'Materials': ' \hspace{3mm}Materials',
     'Consumer parts': ' \hspace{6mm}Consumer Parts',
     'Equipment parts': ' \hspace{6mm}Equipment Parts',
     'Chemical materials': ' \hspace{6mm}Chemical Materials',
     'Energy materials': '\hspace{-3mm} \cbox{blue!70!violet} Energy Materials'}

table = contrib[n.keys()].iloc[-3:].iloc[::-1].T
table.columns = [dtxt(date)['mon6'] for date in table.columns]
table[dtxt(data.index[-13])['mon6']] = contrib[n.keys()].iloc[-13]
table2 = growth[n.keys()].iloc[-3:].iloc[::-1].T
table2.columns = [' ' + dtxt(date)['mon6'] for date in table2.columns]
table2[' '+dtxt(data.index[-13])['mon6']] = growth[n.keys()].iloc[-13]
table = table.join(table2)

table = table.applymap('{:,.1f}'.format)
table.index = [n[name] for name in table.index]

(table.to_csv(data_dir / 'indpro.tex', sep='&', lineterminator='\\\ ', 
              quotechar=' '))

# csv version for download
table2 = table
table2.index = [i[-1] for i in table.index.str.split('}')]
table2.columns = [f'{col} contrib' if i in [0, 1, 2, 3] 
                  else f'{col} growth rate' for i, col 
                  in enumerate(table2.columns)]
table2.to_csv(data_dir / 'indpro_table.csv', index_label='name')

### Bar Chart

In [11]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G17&series=644452cb9b9f8c5a43cd9afb772f1b16&lastobs=50&'
dt = 'from=&to=&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, data = clean_fed_data(url)

ltdate = dtxt(data.index[-1])['mon1']
write_txt(text_dir / 'ip_ind_ldate.txt', ltdate)

df = pd.DataFrame()
for term, name in [('IP', 'IP'), ('CAP.', 'CP')]:
    keys = [key for key in data.keys() if term in key]
    td = (data[keys].iloc[-1] / data[keys].loc['2020-02-01']) - 1
    td.index = td.index.map(d)
    df[name] = td * 100
    
final = df.sort_values('IP', ascending=False)
final.index = [i.replace('products', 'product')
                .replace('product', 'products')
                .replace('eq.', 'equipment') for i in final.index]
table = final.copy().round(1)
table.index = [i.replace('and', '\&').title()
                .replace('Beverage', '\\mbox{Beverage}')
                .replace('Vehicles', '\\mbox{Vehicles}')
                .replace('Electronic', '\\mbox{Electronic}')
                .replace('Rubber', '\\mbox{Rubber}')
                .replace('Transportation', 'Transport.')
                .replace('Miscellaneous', 'Misc.')
                .replace('Equipment', 'Equip.') for i in table.index]
table.to_csv(data_dir / 'ip_comp.csv', index_label='name', sep=';')
words = ['none', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 
         'eight', 'nine', 'ten', '11', 'all 12']
nums = list(range(0, 13))
nums_to_words = dict(zip(nums, words))
verb = {n: 'were' if n!= 1 else 'was' for n in range(0, 13)}

In [12]:
thresh = 0.1
incval = len(final[final['IP'] >= thresh])
decval = len(final[final['IP'] <= -thresh])
unchval = len(final[(final['IP'] < thresh) & (final['IP'] > -thresh)])
and1 = ' and' if unchval == 0 else ''
unctxt = (f', and {nums_to_words[unchval]} {verb[unchval]} unchanged' 
          if unchval > 0 else '')

text0 = (f'As of {ltdate}, of a subset of 12 industries that contribute '+
         f'the majority of industrial production, {nums_to_words[incval]} '+
         'increased \\textbf{production} since February 2020,'+
         f'{and1} {nums_to_words[decval]} decreased production{unctxt} '+
         '(see\cbox{black!20}).\n\n')

largest = final.IP[abs(final.IP).sort_values(ascending=False).iloc[:4].index]
n = {}
i = 0
for name, value in largest.items():
    iname = f'production of {name}' if 'product' in name else f'{name} production'
    inc_dec1 = inc_dec_percent(value)
    inc_dec2 = inc_dec_percent(value, 'of')
    n[i] = (f'{iname} {inc_dec1}'.lower())
    i += 1
text1 = (f'Since February 2020, {n[0]}, {n[1]}, {n[2]}, and {n[3]}. \n\n')

incval = len(final[final['CP'] >= thresh])
decval = len(final[final['CP'] <= -thresh])
unchval = len(final[(final['CP'] < thresh) & (final['CP'] > -thresh)])

text2 = (f'Since February 2020, {nums_to_words[incval]} of the 12 '+
         f'industries increased \\textbf{{capacity}}, {nums_to_words[decval]} '+
         f'decreased capacity, and {nums_to_words[unchval]} {verb[unchval]} unchanged '+
         '(see\cbox{cyan!80!blue}). ')

largest = final.CP[abs(final.CP).sort_values(ascending=False).iloc[:3].index]
n = {}
i = 0
for name, value in largest.items():
    iname = f'production capacity for {name}' if 'product' in name else f'{name} capacity'
    inc_dec1 = inc_dec_percent(value)
    inc_dec2 = inc_dec_percent(value, 'of')
    n[i] = (f'{iname} {inc_dec1}'.lower())
    i += 1
    
text3 = (f'{n[0].capitalize()}, {n[1]}, and {n[2]}.')

end_text = text0 + text1 + text2 + text3
write_txt(text_dir / 'ip_comp.txt', end_text)
print(end_text)

As of October 2023, of a subset of 12 industries that contribute the majority of industrial production, six increased \textbf{production} since February 2020, and six decreased production (see\cbox{black!20}).

Since February 2020, aerospace and miscellaneous transportation equipment production increased by 13.8 percent, paper production decreased by 11.8 percent, motor vehicles and parts production decreased by 8.7 percent, and production of computer and electronic products increased by 8.0 percent. 

Since February 2020, six of the 12 industries increased \textbf{capacity}, five decreased capacity, and one was unchanged (see\cbox{cyan!80!blue}). Production capacity for computer and electronic products increased by 13.9 percent, mining capacity decreased by 12.3 percent, and electric and gas utilities capacity increased by 10.9 percent.


### Capacity Utilization

In [13]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G17&series=316680f2d5251c61c995df7ae36b4b07&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2023&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth
d, data = clean_fed_data(url)
df = data.rename(d, axis=1)
df.to_csv(data_dir / 'tcu.csv', index_label='date')

In [14]:
df = pd.read_csv(data_dir / 'tcu.csv', 
                 index_col='date', parse_dates=True)
adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'Manufacturing': 'blue!40!cyan', 
          'Total index': 'blue!80!black'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'tcu_nodes.txt', nodes)  
ltd = df.iloc[-1]
ltdt = dtxt(df.index[-1])['mon1']
lval = ltd['Total index']

cyr = '2019'
v19 = df.loc[cyr].mean()
dfc = v19.apply('{:.1f} percent'.format) 

sval = df.loc['1989', 'Total index'].mean()
ch19 = value_text(lval - v19['Total index'], 'increase_by', ptype='pp')
totch = value_text(lval - sval, 'increase_by', ptype='pp')
text = (f'In {ltdt}, the US is utilizing {lval:.1f} percent of total '+
        f'industrial capacity {c_line(colors["Total index"])}, and '+
        f'{df["Manufacturing"].iloc[-1]:.1f} percent of manufacturing '+
        f'capacity {c_line(colors["Manufacturing"])}. In {cyr}, the '+
        f'total capacity utilization rate averaged {dfc["Total index"]}, '+
        'and the manufacturing capacity utilization rate averaged '+
        f'{dfc["Manufacturing"]}. Total capacity utilization has '+
        f'{ch19} since {cyr}, and {totch} since 1989.')
write_txt(text_dir / 'tcu.txt', text)
print(text)

In October 2023, the US is utilizing 78.9 percent of total industrial capacity (see {\color{blue!80!black}\textbf{---}}), and 77.2 percent of manufacturing capacity (see {\color{blue!40!cyan}\textbf{---}}). In 2019, the total capacity utilization rate averaged 78.6 percent, and the manufacturing capacity utilization rate averaged 77.1 percent. Total capacity utilization has increased by 0.3 percentage point since 2019, and decreased by 4.8 percentage points since 1989.
